In [1]:
import gc
import os
import random as rn

from itertools import product

import pandas as pd
import numpy as np

import tensorflow as tf
from keras import backend as K
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.client import device_lib

import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
np.random.seed(35)
rn.seed(35)
tf.random.set_seed(35)
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2043672040979678612
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 9389998080
locality {
  bus_id: 1
  links {
  }
}
incarnation: 15284585552535810430
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:65:00.0, compute capability: 7.5"
xla_global_id: 416903419
]


In [3]:
file_path = "../../data/BG Ex data/"
model_path = "../../model/"
file_list = os.listdir(file_path)

In [4]:
fold_info = np.load("../../data/BG_Fold_Info.npz", allow_pickle=True)

In [5]:
train_files = []
test_files = []

for i in range(len(fold_info["train"])):
    train_files.append(fold_info["train"][i])
    test_files.append(fold_info["test"][i])

In [16]:
path = "../../data/BG Naive/"
if not os.path.exists(path):
    os.mkdir(path)

for FOLD in range(10):
    train_data = []
        
    for file in train_files[FOLD]:
        y12 = np.load(file_path+file+".npz")["y12"]
        row = pd.DataFrame({'file_name':file, 'y12':y12})
        train_data.append(row)
        
    train_data = pd.concat(train_data, axis=0)
    train_data.to_csv(path+"Train_Fold"+str(FOLD)+".csv", index=False)
            
    test_data = []
        
    for file in test_files[FOLD]:
        y12 = np.load(file_path+file+".npz")["y12"]
        row = pd.DataFrame({'file_name':file, 'y12':y12})
        test_data.append(row)
        
    test_data = pd.concat(test_data, axis=0)
    test_data.to_csv(path+"Test_Fold"+str(FOLD)+".csv", index=False)

In [6]:
param12 = {
    "FOLD_NUM": [i for i in range(10)],
    "SEQ_LEN": [12],
    "BATCH_SIZE": [128, 256],
    "HIDDEN": [12, 24, 36]
}

param24 = {
    "FOLD_NUM": [i for i in range(10)],
    "SEQ_LEN": [24],
    "BATCH_SIZE": [128, 256],
    "HIDDEN": [24, 48, 72]
}

param36 = {
    "FOLD_NUM": [i for i in range(10)],
    "SEQ_LEN": [36],
    "BATCH_SIZE": [128, 256],
    "HIDDEN": [36, 72, 108]
}

In [7]:
def expand_grid(dict_):
    return pd.DataFrame([row for row in product(*dict_.values())], columns=dict_.keys())

In [8]:
params = pd.concat([expand_grid(param12), expand_grid(param24), expand_grid(param36)])
params.reset_index(inplace=True)
params.drop("index", axis=1, inplace=True)
params

,FOLD_NUM,SEQ_LEN,BATCH_SIZE,HIDDEN
0,0,12,128,12
1,0,12,128,24
2,0,12,128,36
3,0,12,256,12
4,0,12,256,24
...,...,...,...,...
175,9,36,128,72
176,9,36,128,108
177,9,36,256,36
178,9,36,256,72


In [9]:
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

In [10]:
class Model_predict:
    
    def __init__(self, FOLD):
        
        self.train_x = None
        self.train_y = None
        
        self.test_x = None
        self.test_y = None
        
        self.FOLD = FOLD
        self.fold_params = None
        
    def __del__(self):
        
        print("Remove Success\n")
        
    def Add_data(self):
        
        train_data = []
        train_12x, train_24x, train_36x = [],[],[]
        train_y = []
        
        for file in train_files[self.FOLD]:
            train_data.append(np.load(file_path+file+".npz"))
            
        for i in range(len(train_data)):
            train_12x.append(train_data[i]["gen12"])
            train_24x.append(train_data[i]["gen24"])
            train_36x.append(train_data[i]["gen36"])
            train_y.append(train_data[i]["y12"])
            
        test_data = []
        test_12x, test_24x, test_36x = [],[],[]
        test_y = []
        
        for file in test_files[self.FOLD]:
            test_data.append(np.load(file_path+file+".npz"))
            
        for i in range(len(test_data)):
            test_12x.append(test_data[i]["gen12"])
            test_24x.append(test_data[i]["gen24"])
            test_36x.append(test_data[i]["gen36"])
            test_y.append(test_data[i]["y12"])
            
        train_12x = np.concatenate(train_12x).reshape(-1, 12, 1)
        train_24x = np.concatenate(train_24x).reshape(-1, 24, 1)
        train_36x = np.concatenate(train_36x).reshape(-1, 36, 1)
        train_x = {
            "train_12x":train_12x,
            "train_24x":train_24x,
            "train_36x":train_36x
        }
        self.train_x = train_x
        self.train_y = np.concatenate(train_y).reshape(-1, 1)
        
        test_12x = np.concatenate(test_12x).reshape(-1, 12, 1)
        test_24x = np.concatenate(test_24x).reshape(-1, 24, 1)
        test_36x = np.concatenate(test_36x).reshape(-1, 36, 1)
        test_x = {
            "test_12x":test_12x,
            "test_24x":test_24x,
            "test_36x":test_36x
        }
        self.test_x = test_x
        self.test_y = np.concatenate(test_y).reshape(-1, 1)
        
    def Add_param(self):
        
        fold_params_df = params[params["FOLD_NUM"]==self.FOLD]
        fold_params_df.reset_index(inplace=True)
        fold_params_df.drop("index", axis=1, inplace=True)
        
        self.fold_params = fold_params_df
        
    def Prediction(self):
        
        train_list = []
        test_list = []
        df_names = []
        
        for i in range(len(self.fold_params)):
            SEQ = self.fold_params.SEQ_LEN[i]
            HIDDEN = self.fold_params.HIDDEN[i]
            BATCH = self.fold_params.BATCH_SIZE[i]
            
            RNN = keras.models.load_model(model_path+"Fold"+str(self.FOLD)+"/RNN_"+str(SEQ)+"_"+str(HIDDEN)+"_"+str(BATCH)+".h5",
                                          custom_objects={"root_mean_squared_error": root_mean_squared_error})
            train_list.append(RNN.predict(self.train_x["train_"+str(SEQ)+"x"]))
            test_list.append(RNN.predict(self.test_x["test_"+str(SEQ)+"x"]))
            df_names.append("RNN_"+str(SEQ)+"_"+str(HIDDEN)+"_"+str(BATCH))
            
            LSTM = keras.models.load_model(model_path+"Fold"+str(self.FOLD)+"/LSTM_"+str(SEQ)+"_"+str(HIDDEN)+"_"+str(BATCH)+".h5",
                                           custom_objects={"root_mean_squared_error": root_mean_squared_error})
            train_list.append(LSTM.predict(self.train_x["train_"+str(SEQ)+"x"]))
            test_list.append(LSTM.predict(self.test_x["test_"+str(SEQ)+"x"]))
            df_names.append("LSTM_"+str(SEQ)+"_"+str(HIDDEN)+"_"+str(BATCH))
            
            STACKLSTM = keras.models.load_model(model_path+"Fold"+str(self.FOLD)+"/STACKLSTM_"+str(SEQ)+"_"+str(HIDDEN)+"_"+str(BATCH)+".h5",
                                                custom_objects={"root_mean_squared_error": root_mean_squared_error})
            train_list.append(STACKLSTM.predict(self.train_x["train_"+str(SEQ)+"x"]))
            test_list.append(STACKLSTM.predict(self.test_x["test_"+str(SEQ)+"x"]))
            df_names.append("STACKLSTM_"+str(SEQ)+"_"+str(HIDDEN)+"_"+str(BATCH))
            
            BILSTM = keras.models.load_model(model_path+"Fold"+str(self.FOLD)+"/BILSTM_"+str(SEQ)+"_"+str(HIDDEN)+"_"+str(BATCH)+".h5",
                                             custom_objects={"root_mean_squared_error": root_mean_squared_error})
            train_list.append(BILSTM.predict(self.train_x["train_"+str(SEQ)+"x"]))
            test_list.append(BILSTM.predict(self.test_x["test_"+str(SEQ)+"x"]))
            df_names.append("BILSTM_"+str(SEQ)+"_"+str(HIDDEN)+"_"+str(BATCH))
            
            GRU = keras.models.load_model(model_path+"Fold"+str(self.FOLD)+"/GRU_"+str(SEQ)+"_"+str(HIDDEN)+"_"+str(BATCH)+".h5",
                                          custom_objects={"root_mean_squared_error": root_mean_squared_error})
            train_list.append(GRU.predict(self.train_x["train_"+str(SEQ)+"x"]))
            test_list.append(GRU.predict(self.test_x["test_"+str(SEQ)+"x"]))
            df_names.append("GRU_"+str(SEQ)+"_"+str(HIDDEN)+"_"+str(BATCH))
            
            print(str(SEQ)+"_"+str(HIDDEN)+"_"+str(BATCH))
            
        train_list.append(self.train_y)
        test_list.append(self.test_y)
        df_names.append("label")
        
        train_pred_df = pd.DataFrame(np.concatenate(train_list, axis=1), columns=df_names)
        test_pred_df = pd.DataFrame(np.concatenate(test_list, axis=1), columns=df_names)
        
        train_pred_df.to_csv("../../data/BG Pred/Train_Fold"+str(self.FOLD)+"_pred.csv", index=False)
        test_pred_df.to_csv("../../data/BG Pred/Test_Fold"+str(self.FOLD)+"_pred.csv", index=False)
        
        print("Fold"+str(self.FOLD)+" Success !")

In [11]:
directory = "../../data/BG Pred/"

if not os.path.exists(directory):
    os.makedirs(directory)

for i in range(10):
    ML = Model_predict(FOLD = i)
    
    ML.Add_data()
    ML.Add_param()
    ML.Prediction()
    
    del(ML)
    gc.collect()

C:\Users\user\AppData\Local\Temp\ipykernel_59344\496921402.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fold_params_df.drop("index", axis=1, inplace=True)


12_12_128
12_24_128
12_36_128
12_12_256
12_24_256
12_36_256
24_24_128
24_48_128
24_72_128
24_24_256
24_48_256
24_72_256
36_36_128
36_72_128
36_108_128
36_36_256
36_72_256
36_108_256
Fold0 Success !
Remove Success



C:\Users\user\AppData\Local\Temp\ipykernel_59344\496921402.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fold_params_df.drop("index", axis=1, inplace=True)


12_12_128
12_24_128
12_36_128
12_12_256
12_24_256
12_36_256
24_24_128
24_48_128
24_72_128
24_24_256
24_48_256
24_72_256
36_36_128
36_72_128
36_108_128
36_36_256
36_72_256
36_108_256
Fold1 Success !
Remove Success



C:\Users\user\AppData\Local\Temp\ipykernel_59344\496921402.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fold_params_df.drop("index", axis=1, inplace=True)


...

12_12_128
12_24_128
12_36_128
12_12_256
12_24_256
12_36_256
24_24_128
24_48_128
24_72_128
24_24_256
24_48_256
24_72_256
36_36_128
36_72_128
36_108_128
36_36_256
36_72_256
36_108_256
Fold7 Success !
Remove Success



C:\Users\user\AppData\Local\Temp\ipykernel_59344\496921402.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fold_params_df.drop("index", axis=1, inplace=True)


12_12_128
12_24_128
12_36_128
12_12_256
12_24_256
12_36_256
24_24_128
24_48_128
24_72_128
24_24_256
24_48_256
24_72_256
36_36_128
36_72_128
36_108_128
36_36_256
36_72_256
36_108_256
Fold8 Success !
Remove Success



C:\Users\user\AppData\Local\Temp\ipykernel_59344\496921402.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fold_params_df.drop("index", axis=1, inplace=True)


12_12_128
12_24_128
12_36_128
12_12_256
12_24_256
12_36_256
24_24_128
24_48_128
24_72_128
24_24_256
24_48_256
24_72_256
36_36_128
36_72_128
36_108_128
36_36_256
36_72_256
36_108_256
Fold9 Success !
Remove Success

